In [97]:
import urllib
import urllib.request
import os
import re

In [98]:
dir1 = "GamesPGN/"
dir2 = "IndividualGames/"
dir3 = "LegibleMoves/"

for f in os.listdir(dir1):
    if ".ipynb" in f:
        continue
    os.remove(dir1 + f)
    
for f in os.listdir(dir2):
    if ".ipynb" in f:
        continue
    os.remove(dir2 + f)
    
for f in os.listdir(dir3):
    if ".ipynb" in f:
        continue
    os.remove(dir3 + f)

In [99]:
usernameList = [
'SenseiDanya',
'OhMyLands',
'FrankfurtAirport',
]


for username in usernameList:

    baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"
    archivesUrl = baseUrl + "archives"

    #read the archives url and store in a list
    f = urllib.request.urlopen(archivesUrl)
    archives = f.read().decode("utf-8")
    archives = archives.replace("{\"archives\":[", "")
    archives = archives.replace("\"]}", "")
    archives = archives.replace("\"", "")
    archivesList = archives.split(',')
    archivesList = [i for i in archivesList]

    #download all the archives
    for i, name in enumerate(archivesList):
        
        url = name + "/pgn"
        filename = username + str(i)
        urllib.request.urlretrieve(url, dir1 +  filename + ".pgn")


In [100]:
startGame = "[Event \"Live Chess\"]\n"

files = os.listdir(dir1)
gameNumber = 0

skipGame = False

for f in files:
    
    if ".ipynb" in f:
        continue
        
    with open(dir1 + f, 'r') as infile:
        next(infile) #Skip the first header so we do not save
        game = startGame #Manually add the line we skipped
        
        for line in infile:
            
            if "UTCDate" in line:
                plchldr = line
                plchldr = plchldr.strip('[')
                plchldr = plchldr.strip(']\n')
                plchldr = plchldr.split(' ')
                date = plchldr[-1][1:-1]
            if "UTCTime" in line:
                plchldr = line
                plchldr = plchldr.strip('[')
                plchldr = plchldr.strip(']\n')
                plchldr = plchldr.split(' ')
                time = plchldr[-1][1:-1]
            
            
            
            if "TimeControl" in line:
                plchldr = line
                plchldr = plchldr.strip('[')
                plchldr = plchldr.strip(']\n')
                plchldr = plchldr.split(' ')
                
                timeGame = plchldr[-1][1:-1] 
                if timeGame not in ["300","600", "600+5", "900", "900+10"]:
                    skipGame = True
                    print("Will skip game with time control ", timeGame)
            
            if line == startGame:
                if skipGame: 
                    skipGame = False
                    game = ""
                    continue
                    
                with open( dir2 + (date + '_' +  time).replace('.', '_').replace(':', '_'), 'w') as outfile:
                    outfile.write(game)
                gameNumber +=1
                game = ""
            
            game += line
            
        
        if skipGame: 
            skipGame = False
            game = ""
            continue
            
        with open( dir2 + (date + '_' + time).replace('.', '_').replace(':', '_'), 'w') as outfile:
            outfile.write(game)
            gameNumber +=1
            game = ""        
        

Will skip game with time control  60
Will skip game with time control  180+2
Will skip game with time control  180+2
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  30
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180


In [101]:
pieces = {
    'R' : 'rook ',
    'N' : 'knight ',
    'B' : 'bishop ',
    'K' : 'king ',
    'Q' : 'queen '
}

moves = {
    'x'    : ' takes on '
}

In [102]:
def MakeGameLegible(game):

    mod = re.sub('{.+?}', '', game)
    mod2 = ""
    for letter in mod:
        if letter in list(pieces.keys()):
            mod2 += pieces[letter]

        elif letter in list(moves.keys()):
            mod2 += moves[letter]
        else:
            mod2 += letter

    mod3 = re.sub('O-O-O', 'long castle ', mod2)    
    mod4 = re.sub('O-O', 'castle ', mod3)    
    mod5 = re.sub('1-0\n', 'white wins', mod4) 
    mod6 = re.sub('0-1\n', 'black wins', mod5)
    mod7 = re.sub('1/2-1/2\n', 'draw', mod6)
    
    return(mod7)

In [103]:
startGame = "[Event \"Live Chess\"]\n"

files = os.listdir(dir1)
gameNumber = 0

skipGame = False

for f in files:
    
    if ".ipynb" in f:
        continue
        
    with open(dir1 + f, 'r') as infile:        
        for line in infile:
            
            
            if "UTCDate" in line:
                plchldr = line
                plchldr = plchldr.strip('[')
                plchldr = plchldr.strip(']\n')
                plchldr = plchldr.split(' ')
                date = plchldr[-1][1:-1]
            if "UTCTime" in line:
                plchldr = line
                plchldr = plchldr.strip('[')
                plchldr = plchldr.strip(']\n')
                plchldr = plchldr.split(' ')
                time = plchldr[-1][1:-1]
            
            if "TimeControl" in line:
                plchldr = line
                plchldr = plchldr.strip('[')
                plchldr = plchldr.strip(']\n')
                plchldr = plchldr.split(' ')
                
                timeGame = plchldr[-1][1:-1] 
                if timeGame not in ["300","600", "600+5", "900", "900+10"]:
                    skipGame = True
                    print("Will skip game with time control ", timeGame)
            
            if line[0:2] == "1.":
                game = line
                if skipGame: 
                    skipGame = False
                    continue
                    
                with open( dir3 + (date + '_' + time).replace('.', '_').replace(':', '_'), 'w') as outfile:
                    outfile.write(MakeGameLegible(line))
                gameNumber +=1
            

Will skip game with time control  60
Will skip game with time control  180+2
Will skip game with time control  180+2
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  30
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
Will skip game with time control  180
